In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import urllib
import requests
import time
import japanize_matplotlib
from sklearn.metrics import mean_squared_error
from scipy import stats as st
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import LabelEncoder
import pickle
import os
import lightgbm as lgb
import unicodedata
from sklearn.model_selection import train_test_split,KFold,GroupKFold

In [3]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
card=pd.read_csv("card.csv")
user=pd.read_csv("user.csv")
sub=pd.read_csv("sample_submit.csv")

In [9]:
train

,index,user_id,card_id,amount,errors?,is_fraud?,merchant_id,merchant_city,merchant_state,zip,mcc,use_chip
0,0,1721,0,$2.623,OK,0,209237,Joliet,IL,60436.0,5541,Swipe Transaction
1,1,1629,3,$6.4,OK,0,2568,Edgerton,WI,53534.0,5814,Swipe Transaction
2,2,655,3,$123.5,OK,0,345310,Ridgefield,WA,98642.0,7538,Swipe Transaction
3,3,492,0,$51.287,OK,0,4295,Milton,FL,32583.0,5912,Chip Transaction
4,4,1969,4,$17.561,OK,0,350447,Irvington,NJ,7111.0,4214,Swipe Transaction
...,...,...,...,...,...,...,...,...,...,...,...,...
471278,471278,1572,0,$51.245,OK,0,325177,Lake Orion,MI,48362.0,5541,Chip Transaction
471279,471279,837,2,$95.717,OK,1,405215,Norcross,GA,30092.0,7922,Chip Transaction
471280,471280,1522,1,$49.0,OK,0,291280,Novato,CA,94947.0,5411,Swipe Transaction
471281,471281,1328,1,$22.868,OK,0,312028,ONLINE,NaN,NaN,7349,Online Transaction


In [8]:
card

,user_id,card_id,card_brand,card_type,expires,has_chip,cards_issued,credit_limit,acct_open_date,year_pin_last_changed
0,39,0,Visa,Debit,09/2021,YES,1,$17117,05/2007,2010
1,39,1,Amex,Credit,11/2024,YES,2,$5400,10/2015,2015
2,41,0,Discover,Credit,03/2022,YES,2,$14800,12/2010,2011
3,41,1,Mastercard,Debit,09/2024,YES,2,$4183,09/2010,2010
4,41,2,Mastercard,Debit,03/2024,YES,1,$6732,02/2010,2010
...,...,...,...,...,...,...,...,...,...,...
411,1969,6,Mastercard,Debit,11/2021,YES,1,$11647,08/2017,2017
412,1986,0,Visa,Debit,10/2021,YES,2,$35649,06/2006,2011
413,1986,1,Visa,Debit,10/2012,YES,2,$18979,03/2004,2008
414,1986,2,Mastercard,Credit,02/2023,NO,2,$10000,12/2006,2012


In [7]:
user

,user_id,current_age,retirement_age,birth_year,birth_month,gender,address,city,state,zipcode,latitude,longitude,per_capita_income_zipcode,yearly_income_person,total_debt,fico_score,num_credit_cards
0,39,57,64,1962,12,Female,442 Burns Boulevard,Mansfield,MA,2048,42.02,-71.21,$37407,$76274,$102611,698,2
1,41,39,66,1980,10,Female,3863 River Avenue,Lincoln,CA,95648,38.93,-121.25,$21829,$44506,$57994,849,3
2,47,40,67,1979,5,Female,8799 Elm Avenue,Mckinney,TX,75069,33.20,-96.65,$24684,$50329,$76759,625,4
3,54,59,67,1960,3,Male,1362 Norfolk Drive,Montgomery,AL,36111,32.35,-86.28,$21102,$43021,$86014,725,4
4,80,66,59,1954,1,Female,1105 Second Drive,Dearborn,MI,48124,42.31,-83.21,$24561,$48690,$36262,701,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1799,36,67,1983,12,Male,42720 East Drive,Mukwonago,WI,53149,42.86,-88.33,$28215,$57532,$61268,735,4
93,1833,76,71,1943,8,Female,172 Birch Street,Flagstaff,AZ,86004,35.29,-111.32,$21950,$27484,$16803,660,4
94,1947,78,66,1941,9,Male,6283 Rose Avenue,Chantilly,VA,20152,38.87,-77.40,$46175,$113514,$16524,727,8
95,1969,59,68,1960,3,Female,5238 Park Street,Union City,NJ,7087,40.76,-74.03,$16770,$34190,$39242,810,7
